In [ ]:
import os
import pickle

def load_all_pickles(directory_path):
    # Dictionary to store the combined data from all pickle files
    output_data = {}

    # Check if the directory exists
    if not os.path.exists(directory_path):
        print(f"Directory {directory_path} not found.")
        return output_data

    # List all files in the directory
    for file_name in os.listdir(directory_path):
        # Check if the file is a pickle file
        if file_name.endswith('.pickle'):
            file_path = os.path.join(directory_path, file_name)
            try:
                with open(file_path, 'rb') as file:
                    # Load the pickle file and extend the output dictionary
                    file_data = pickle.load(file)
                    output_data.update(file_data)
            except Exception as e:
                print(f"Error reading file {file_name}: {e}")

    return output_data

In [ ]:

# Load all pickle files from the data directory and combine them into a single dictionary
directory_path = 'data'
combined_data = load_all_pickles(directory_path)

In [ ]:
profiles = []

In [ ]:
from copy import deepcopy

In [ ]:
for key, value in combined_data.items():
    timestamp = key[2]
    position = key[:2]
    for profile in value['items']:
        prof = {}
        prof.update(profile['data'])
        prof['scraped_at'] = timestamp
        prof['position'] = position

        profiles.append(deepcopy(prof))

In [ ]:
import pandas as pd

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(profiles)

# Display the first few rows to confirm
print(df.head())

In [ ]:
import math
import localization as lx
# Initialize an empty dictionary to store the results
location_data = {}
# Group the DataFrame by 'profileId'
grouped = df.groupby('profileId')

P=lx.Project(mode='Earth1',solver='LSE')
targets = []
for profileId, group in grouped:
    # Extract reference points and distances for the current group
    ref_points = group['position'].tolist()
    distances = group['distanceMeters'].tolist()
    # print(ref_points, distances)
    if len(ref_points) < 3:
        continue
    if any(math.isnan(distance) for distance in distances):
        continue

    for i in range(len(ref_points)):
        P.add_anchor(f'anchore_{i}_{profileId}',ref_points[i])
    t,label=P.add_target(ID=profileId)
    targets.append(t)
    for i in range(len(distances)):
        t.add_measure(f'anchore_{i}_{profileId}',distances[i])

    location_data[profileId] = {'ref_points':ref_points, 'distances':distances}



In [ ]:
P.solve()

In [ ]:
location_data.keys()

In [ ]:
for t in targets:
    if float(t.ID) in list(location_data.keys()):
        location_data[float(t.ID)]['estimated_position'] = t.loc.x, t.loc.y

In [ ]:
# Display the estimated positions
for profileId, position in location_data.items():
    print(f"Profile ID: {profileId}, Estimated Position: {position.get('estimated_position', 'N/A')}")

In [ ]:
testID = 612839260.0

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Example reference points and distances (lat, lon) and meters
reference_points = location_data[testID]['ref_points']
distances = location_data[testID]['distances']
calculated_point = location_data[testID]['estimated_position']

# Plotting
fig, ax = plt.subplots()

# Convert distances to degrees approximately (quick approximation, valid for short distances)
# Assuming a rough conversion factor (not accurate for large distances or near the poles)
distance_degrees = [d / 111139 for d in distances]

# Plot reference points
for (lat, lon), distance_degree in zip(reference_points, distance_degrees):
    ax.plot(lon, lat, 'bo')  # Reference points in blue
    circle = plt.Circle((lon, lat), distance_degree, color='b', fill=False, linestyle='--')
    ax.add_artist(circle)

# Plot the calculated point
ax.plot(calculated_point[1], calculated_point[0], 'rx')  # Calculated point in red

# Set labels and show plot
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('Multilateration Visualization')
plt.grid(True)
plt.axis('equal')  # Equal aspect ratio to ensure circles look like circles
plt.show()
